In [1]:
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
import random
import datetime
%load_ext tensorboard
from google.cloud import datastore
import os
import numpy as np
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = \
    "/home/alice/Downloads/HCL-customer-application-c05a56fa9ccc.json"

client = datastore.Client()


# Load title and Subject data for prediction

In [2]:
query = client.query(kind="tododata")
x = []
y = []
z = []
for i in query.fetch():
    x.append([i['subject'], i['title']])
    y.append([i['totalcoins']])
    z.append(i['prev'])
y = np.asarray(y)


In [3]:
max_length = max([len(i) for i in z])
max_length
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.feature_extraction.text import CountVectorizer
title = CountVectorizer(min_df=0, lowercase=False)
title.fit([item['title'] for span in z for item in span])
subject = CountVectorizer(min_df=0, lowercase=False)
subject.fit([item['subject'] for span in z for item in span])
data = []
for i in z:
    titles = title.transform([ent['title'] for ent in i]).toarray()
    subjects = subject.transform([ent['subject'] for ent in i]).toarray()
    time = np.asarray([[ent['time']] for ent in i]).reshape(-1,1)
    dat = np.concatenate((titles, subjects,time), axis=1)
    data.append(dat)
    
prev = pad_sequences(data,padding='post')

convert both title and subject to one hot encoded matrices because the datasets are both categorical and orthogonal to each other

In [4]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(min_df=0, lowercase=False)

In [5]:
def toCategorical(data):
    vectorizer.fit(data)
    return vectorizer.transform(data).toarray()

In [6]:
title = toCategorical([i[1] for i in x])

In [7]:
subjects = toCategorical([i[0] for i in x])

In [8]:
x = np.concatenate((title,subjects),axis=1)


## normalize the prediction values to aid the neural net in fitting the data

In [9]:
vec_train, vec_test, y_train, y_test,prev_train ,prev_test = train_test_split(x, y,prev, test_size=0.2)

In [10]:
print(vec_train.shape)
print(prev_train.shape)
print(y_train.shape)

(43, 62)
(43, 25, 60)
(43, 1)


In [24]:
from tensorflow.keras import layers
from tensorflow.keras.layers import Input, Concatenate, GRU, Flatten, Dense
from tensorflow.keras.models import Model

prev_input = Input((prev.shape[1],prev.shape[2],))
vector_input = Input((x.shape[1],))



conv_layer = GRU(3)(prev_input)
flat_layer = Flatten()(conv_layer)

concat_layer = Concatenate()([vector_input, flat_layer])
output = Dense(8)(concat_layer)
coins = Dense(1)(output)

model = Model(inputs=[vector_input,prev_input],outputs=coins)

model.compile(optimizer=tf.keras.optimizers.Adam(0.01),
              loss='mse',       # mean squared error
              metrics=['mae'])  # mean absolute error
print(model.summary())

Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_9 (InputLayer)            [(None, 25, 60)]     0                                            
__________________________________________________________________________________________________
gru_4 (GRU)                     (None, 3)            585         input_9[0][0]                    
__________________________________________________________________________________________________
input_10 (InputLayer)           [(None, 62)]         0                                            
__________________________________________________________________________________________________
flatten_4 (Flatten)             (None, 3)            0           gru_4[0][0]                      
____________________________________________________________________________________________

In [25]:
log_dir = "rewardlogs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

model.fit((vec_train,prev_train), y_train, 
          validation_data=((vec_test,prev_test),y_test),
          batch_size=15, 
          epochs=70,
          verbose=1,
         callbacks=[tensorboard_callback])

Epoch 1/70
3/3 [==============================] - 1s 240ms/step - loss: 1530321.5000 - mae: 1161.5016 - val_loss: 1697826.1250 - val_mae: 1239.5679
Epoch 2/70
3/3 [==============================] - 0s 36ms/step - loss: 1528424.0000 - mae: 1160.7073 - val_loss: 1695678.5000 - val_mae: 1238.7667
Epoch 3/70
3/3 [==============================] - 0s 33ms/step - loss: 1526432.3750 - mae: 1159.8442 - val_loss: 1693158.1250 - val_mae: 1237.8267
Epoch 4/70
3/3 [==============================] - 0s 31ms/step - loss: 1524027.0000 - mae: 1158.8483 - val_loss: 1690327.6250 - val_mae: 1236.7651
Epoch 5/70
3/3 [==============================] - 0s 29ms/step - loss: 1521505.5000 - mae: 1157.7349 - val_loss: 1687013.1250 - val_mae: 1235.5186
Epoch 6/70
3/3 [==============================] - 0s 34ms/step - loss: 1518425.5000 - mae: 1156.4382 - val_loss: 1683213.5000 - val_mae: 1234.0808
Epoch 7/70
3/3 [==============================] - 0s 36ms/step - loss: 1514903.7500 - mae: 1154.9364 - val_loss: 1678

Epoch 57/70
3/3 [==============================] - 0s 40ms/step - loss: 759851.8125 - mae: 771.2279 - val_loss: 823982.0000 - val_mae: 839.8162
Epoch 58/70
3/3 [==============================] - 0s 38ms/step - loss: 740549.8125 - mae: 761.0443 - val_loss: 801560.0000 - val_mae: 827.0815
Epoch 59/70
3/3 [==============================] - 0s 32ms/step - loss: 720615.9375 - mae: 750.6723 - val_loss: 779604.3750 - val_mae: 814.4233
Epoch 60/70
3/3 [==============================] - 0s 33ms/step - loss: 702396.8750 - mae: 741.1511 - val_loss: 757533.2500 - val_mae: 801.5068
Epoch 61/70
3/3 [==============================] - 0s 33ms/step - loss: 683253.1875 - mae: 731.1608 - val_loss: 735855.3750 - val_mae: 788.6097
Epoch 62/70
3/3 [==============================] - 0s 39ms/step - loss: 663741.0000 - mae: 721.0296 - val_loss: 714631.7500 - val_mae: 775.7791
Epoch 63/70
3/3 [==============================] - 0s 35ms/step - loss: 646501.0000 - mae: 710.9418 - val_loss: 693210.5000 - val_mae: 7

In [26]:
%tensorboard --logdir rewardlogs/fit


Reusing TensorBoard on port 6007 (pid 373843), started 0:11:57 ago. (Use '!kill 373843' to kill it.)